In [2]:
# import necessary packages
import os                            //
import cv2
import time
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Activation, Dropout

In [3]:
img_size = 100
datadir = r'D:\CVPR\dataset'
CATEGORIES = os.listdir(datadir)
print(CATEGORIES)

['mask', 'no_mask']


In [4]:
# Define two empty list for contain image data
x, y = [], []
   
def PreProcess():
    for category in CATEGORIES:
        path = os.path.join(datadir, category)
        classIndex = CATEGORIES.index(category)
        print(path)
        for imgs in tqdm(os.listdir(path)):
            img_arr = cv2.imread(os.path.join(path, imgs))
            
            # resize the image
            resized_array = cv2.resize(img_arr, (img_size, img_size))
            cv2.imshow("images", resized_array)
            cv2.waitKey(1)
            resized_array = resized_array/255.0
            x.append(resized_array)
            y.append(classIndex)
            
PreProcess()
cv2.destroyAllWindows()

D:\CVPR\dataset\mask


100%|██████████████████████████████████████████████████████████████████████████████| 2994/2994 [00:50<00:00, 59.46it/s]


D:\CVPR\dataset\no_mask


100%|██████████████████████████████████████████████████████████████████████████████| 2994/2994 [00:48<00:00, 61.21it/s]


In [5]:
# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [6]:
# Convert and resize the data to a numpy array
X_train = np.array(X_train).reshape(-1, img_size, img_size, 3)
y_train = np.array(y_train)
X_test = np.array(X_test).reshape(-1, img_size, img_size, 3)
y_test = np.array(y_test)

In [7]:
batch_size = 32
epochs = 15

In [8]:
# Create the model architecture

model = Sequential()

model.add(Conv2D(64,(3, 3), input_shape=(img_size, img_size, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(16, activation='relu'))


model.add(Dense(len(CATEGORIES)))
model.add(Activation('softmax'))

# compile the model

model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 64)        1792      
                                                                 
 activation (Activation)     (None, 98, 98, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 256)       147712    
                                                                 
 activation_1 (Activation)   (None, 47, 47, 256)       0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 256)      0         
 2D)                                                    

In [9]:
t1 = time.time()

# fit the model
model.fit(X_train, y_train, batch_size = batch_size, epochs=5, validation_split=0.3, verbose = 1)
model.save('{}.h5'.format("model"))

t2 = time.time()
print('Time taken: ',t2-t1)

Epoch 1/5
105/105 [==============================] - 90s 838ms/step - loss: 0.2708 - accuracy: 0.8858 - val_loss: 0.1621 - val_accuracy: 0.9436
Epoch 2/5
105/105 [==============================] - 105s 1s/step - loss: 0.1569 - accuracy: 0.9472 - val_loss: 0.1096 - val_accuracy: 0.9666
Epoch 3/5
105/105 [==============================] - 109s 1s/step - loss: 0.1136 - accuracy: 0.9648 - val_loss: 0.1070 - val_accuracy: 0.9589
Epoch 4/5
105/105 [==============================] - 105s 1s/step - loss: 0.1109 - accuracy: 0.9657 - val_loss: 0.1148 - val_accuracy: 0.9680
Epoch 5/5
105/105 [==============================] - 103s 979ms/step - loss: 0.1085 - accuracy: 0.9624 - val_loss: 0.0865 - val_accuracy: 0.9756
Time taken:  517.4427332878113


In [10]:
validation_loss, validation_accuracy = model.evaluate(X_test, y_test)

38/38 [==============================] - 8s 213ms/step - loss: 0.1023 - accuracy: 0.9683


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the model
model = load_model('model.h5')

# Define mediapipe Face detector

face_detection = mp.solutions.face_detection.FaceDetection()

# Detection function



def get_detection(frame):

    height, width, channel = frame.shape

    # Convert frame BGR to RGB colorspace

    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Detect results from the frame
    
    result = face_detection.process(imgRGB)


    try:
        for count, detection in enumerate(result.detections):

            # print(detection)

        
            # Extract bounding box information 
            
            box = detection.location_data.relative_bounding_box

            x, y, w, h = int(box.xmin*width), int(box.ymin * height), int(box.width*width), int(box.height*height)
            
    # If detection is not available then pass 
    except:
        pass

    return x, y, w, h

CATEGORIES = ['no_mask', 'mask']
cap = cv2.VideoCapture(0)
while True:
    _, frame = cap.read()
    img = frame.copy()
    try:
        x, y, w, h = get_detection(frame)
        
        crop_img = img[y:y+h, x:x+w]
        
        crop_img = cv2.resize(crop_img, (100, 100))
        
        crop_img = np.expand_dims(crop_img, axis=0)
        
        # get the prediction from the model.
        prediction = model.predict(crop_img)
        print(prediction)
        index = np.argmax(prediction)
        res = CATEGORIES[index]
        if index == 0:
            color = (0, 0, 255)
        else:
            color = (0, 255, 0)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, res, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX,
                                 0.8, color, 2, cv2.LINE_AA)

    except:
        pass
    
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 1s 1s/step
[[1. 0.]]
1/1 [==============================] - 0s 32ms/step
[[1. 0.]]
1/1 [==============================] - 0s 36ms/step
[[1.000000e+00 6.048963e-11]]
1/1 [==============================] - 0s 34ms/step
[[1.000000e+00 6.048963e-11]]
1/1 [==============================] - 0s 35ms/step
[[6.645968e-06 9.999933e-01]]
1/1 [==============================] - 0s 38ms/step
[[1.000000e+00 5.943576e-14]]
1/1 [==============================] - 0s 33ms/step
[[1.000000e+00 5.943576e-14]]
1/1 [==============================] - 0s 37ms/step
[[7.6725384e-22 1.0000000e+00]]
1/1 [==============================] - 0s 32ms/step
[[0. 1.]]
1/1 [==============================] - 0s 38ms/step
[[0. 1.]]
1/1 [==============================] - 0s 32ms/step
[[0. 1.]]
1/1 [==============================] - 0s 35ms/step
[[0. 1.]]
1/1 [==============================] - 0s 35ms/step
[[0. 1.]]
1/1 [==============================] - 0s 41ms/step
[[0. 1.]]
1/1 [=======

1/1 [==============================] - 0s 39ms/step
[[0. 1.]]
1/1 [==============================] - 0s 34ms/step
[[0. 1.]]
1/1 [==============================] - 0s 33ms/step
[[0. 1.]]
1/1 [==============================] - 0s 33ms/step
[[3.2702538e-23 1.0000000e+00]]
1/1 [==============================] - 0s 36ms/step
[[2.0461915e-08 1.0000000e+00]]
1/1 [==============================] - 0s 35ms/step
[[2.0461915e-08 1.0000000e+00]]
1/1 [==============================] - 0s 37ms/step
[[8.5114266e-10 1.0000000e+00]]
1/1 [==============================] - 0s 34ms/step
[[1.1553515e-05 9.9998844e-01]]
1/1 [==============================] - 0s 34ms/step
[[1.1553515e-05 9.9998844e-01]]
1/1 [==============================] - 0s 34ms/step
[[2.398765e-23 1.000000e+00]]
1/1 [==============================] - 0s 35ms/step
[[0. 1.]]
1/1 [==============================] - 0s 34ms/step
[[2.9891096e-13 1.0000000e+00]]
1/1 [==============================] - 0s 38ms/step
[[2.9891096e-13 1.0000000e+00]

1/1 [==============================] - 0s 34ms/step
[[0. 1.]]
1/1 [==============================] - 0s 38ms/step
[[1.1656589e-14 1.0000000e+00]]
1/1 [==============================] - 0s 33ms/step
[[4.1110365e-19 1.0000000e+00]]
1/1 [==============================] - 0s 34ms/step
[[0. 1.]]
1/1 [==============================] - 0s 32ms/step
[[0. 1.]]
1/1 [==============================] - 0s 34ms/step
[[0. 1.]]
1/1 [==============================] - 0s 35ms/step
[[0. 1.]]
1/1 [==============================] - 0s 33ms/step
[[0. 1.]]
1/1 [==============================] - 0s 34ms/step
[[0. 1.]]
1/1 [==============================] - 0s 33ms/step
[[1.00000e+00 8.14564e-11]]
1/1 [==============================] - 0s 32ms/step
[[1.00000e+00 8.14564e-11]]
1/1 [==============================] - 0s 42ms/step
[[1.0000000e+00 3.2762675e-11]]
1/1 [==============================] - 0s 37ms/step
[[0.9614359  0.03856409]]
1/1 [==============================] - 0s 38ms/step
[[1.0000000e+00 5.210230

[[0. 1.]]
1/1 [==============================] - 0s 40ms/step
[[0. 1.]]
1/1 [==============================] - 0s 41ms/step
[[0. 1.]]
1/1 [==============================] - 0s 42ms/step
[[0. 1.]]
1/1 [==============================] - 0s 42ms/step
[[0. 1.]]
1/1 [==============================] - 0s 40ms/step
[[0. 1.]]
1/1 [==============================] - 0s 38ms/step
[[0. 1.]]
1/1 [==============================] - 0s 39ms/step
[[4.644494e-08 1.000000e+00]]
1/1 [==============================] - 0s 40ms/step
[[0. 1.]]
1/1 [==============================] - 0s 40ms/step
[[0. 1.]]
1/1 [==============================] - 0s 39ms/step
[[0. 1.]]
1/1 [==============================] - 0s 40ms/step
[[0. 1.]]
1/1 [==============================] - 0s 41ms/step
[[0. 1.]]
1/1 [==============================] - 0s 42ms/step
[[5.5858915e-25 1.0000000e+00]]
1/1 [==============================] - 0s 39ms/step
[[1.8006976e-35 1.0000000e+00]]
1/1 [==============================] - 0s 39ms/step
[[1.80

1/1 [==============================] - 0s 42ms/step
[[1. 0.]]
1/1 [==============================] - 0s 44ms/step
[[1. 0.]]
1/1 [==============================] - 0s 50ms/step
[[1. 0.]]
1/1 [==============================] - 0s 49ms/step
[[1. 0.]]
1/1 [==============================] - 0s 49ms/step
[[1. 0.]]
1/1 [==============================] - 0s 48ms/step
[[1. 0.]]
1/1 [==============================] - 0s 54ms/step
[[1. 0.]]
1/1 [==============================] - 0s 55ms/step
[[1. 0.]]
1/1 [==============================] - 0s 53ms/step
[[1. 0.]]
1/1 [==============================] - 0s 47ms/step
[[1. 0.]]
1/1 [==============================] - 0s 48ms/step
[[1. 0.]]
1/1 [==============================] - 0s 53ms/step
[[1. 0.]]
1/1 [==============================] - 0s 53ms/step
[[1. 0.]]
1/1 [==============================] - 0s 48ms/step
[[1. 0.]]
1/1 [==============================] - 0s 51ms/step
[[1. 0.]]
1/1 [==============================] - 0s 53ms/step
[[1. 0.]]
1/1 [===

1/1 [==============================] - 0s 40ms/step
[[0. 1.]]
1/1 [==============================] - 0s 40ms/step
[[0. 1.]]
1/1 [==============================] - 0s 39ms/step
[[0. 1.]]
1/1 [==============================] - 0s 41ms/step
[[0. 1.]]
1/1 [==============================] - 0s 37ms/step
[[0. 1.]]
1/1 [==============================] - 0s 40ms/step
[[0. 1.]]
1/1 [==============================] - 0s 100ms/step
[[0. 1.]]
1/1 [==============================] - 0s 43ms/step
[[0. 1.]]
1/1 [==============================] - 0s 43ms/step
[[0. 1.]]
1/1 [==============================] - 0s 40ms/step
[[0. 1.]]
1/1 [==============================] - 0s 41ms/step
[[0. 1.]]
1/1 [==============================] - 0s 41ms/step
[[0. 1.]]
1/1 [==============================] - 0s 40ms/step
[[0. 1.]]
1/1 [==============================] - 0s 39ms/step
[[0. 1.]]
1/1 [==============================] - 0s 39ms/step
[[0. 1.]]
1/1 [==============================] - 0s 39ms/step
[[0. 1.]]
1/1 [==